In [ ]:
import numpy as np

In [ ]:
def smearMat(myL, sL, sNL):
	nn = np.linspace(0,myL**3-1, num=myL**3)
	nx = nn % myL
	ny = ((nn - nx) / myL) % myL
	nz = (nn-nx-ny*myL) / myL ** 2
	nxp = ((nx+1) % myL) + ny*myL + nz*myL**2
	nxm = ((nx-1) % myL) + ny*myL + nz*myL**2
	nyp = nx + ((ny+1) % myL)*myL + nz*myL**2
	nym = nx + ((ny-1) % myL)*myL + nz*myL**2
	nzp = nx + ny*myL + ((nz+1) % myL)*myL**2
	nzm = nx + ny*myL + ((nz-1) % myL)*myL**2
	
	smear = np.zeros([myL**3, myL**3])

	for i in range(myL**3):
		smear[int(nn[i])][int(nxp[i])] += 1
		smear[int(nn[i])][int(nxm[i])] += 1
		smear[int(nn[i])][int(nyp[i])] += 1
		smear[int(nn[i])][int(nym[i])] += 1
		smear[int(nn[i])][int(nzp[i])] += 1
		smear[int(nn[i])][int(nzm[i])] += 1
		
	id = np.identity(myL**3)
	locMat = sL*smear + id
	nonLocMat = sNL*smear + id
	return locMat, nonLocMat

In [117]:
def interact(myL, c0, sL, sNL):
	local, nonLocal = smearMat(myL, sL, sNL)
	locOrigin = np.zeros([myL**3 , 1])
	locOrigin[0,0] = 1.0
	locSq = np.square(local)
	nonLocSq = np.square(nonLocal)
	locPot = np.dot(locSq, locOrigin)

	fSL = np.zeros([myL**3, myL**3])
	for i in range(myL**3):
		fSL[i,i] = locPot[i][0]

	return c0/2 * np.dot(np.dot(nonLocSq, fSL), nonLocSq)

In [ ]:
print(interact(2, 1, 1, 1))

[[96.5 10.  10.  64.  10.  64.  64.   0. ]
 [10.  10.   8.   8.   8.   8.   0.   0. ]
 [10.   8.  10.   8.   8.   0.   8.   0. ]
 [64.   8.   8.  64.   0.  32.  32.   0. ]
 [10.   8.   8.   0.  10.   8.   8.   0. ]
 [64.   8.   0.  32.   8.  64.  32.   0. ]
 [64.   0.   8.  32.   8.  32.  64.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0.   0. ]]
